## **Project 2: Hyperparameter Optimization**

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten, Conv2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sklearn
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import zipfile
from tensorflow.keras.preprocessing import image
import os

In [2]:
trainDIR = '/workspace/seg_train/seg_train/'
testDIR = '/workspace/seg_test/seg_test/'
predDIR = '/workspace/seg_pred/seg_pred'

In [3]:
def get_images(directory):
    Images=[]
    Labels=[]
    
    for labels in os.listdir(directory):
        if labels == 'glacier':
            label = 0
        elif labels == 'sea':
            label = 1
        elif labels == 'buildings':
            label = 2
        elif labels == 'forest':
            label = 3
        elif labels == 'mountain':
            label = 4
        elif labels == 'street':
            label = 5
        
        for file in os.listdir(directory+labels):
            im =  image.load_img(directory+labels+r'/'+file, target_size=(150,150))
            img = image.img_to_array(im)
            Images.append(img)
            Labels.append(label)
    
    Images, Labels = sklearn.utils.shuffle(Images, Labels, random_state=14)
    return Images, Labels

In [4]:
Images, Labels = get_images(trainDIR)
Images = np.array(Images)
Labels = np.array(Labels)

In [5]:
def create_model(lr=0.01, opt = keras.optimizers.Adam(),
                 nn1 = 32, nn2 = 100):
                                                     
    model = Sequential()
    
    model.add(Convolution2D(nn1,(3,3), input_shape=(150,150,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Flatten())
    model.add(Dense(nn2, activation='relu'))
    model.add(Dropout(0))   
    model.add(Dense(6, activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model


In [7]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# model class to use in the scikit random search CV 
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=5, validation_split=0.25)

In [8]:
# learning rate parameters
lr=[1e-2, 1e-3]

# neurons in each layer
nn1 = [32,64]
nn2 = [100,200]

# optimizer
optimizer = ['adam','rmsprop']

# dictionary summary
param_grid = dict(nn1= nn1, nn2 = nn2, lr=lr, opt=optimizer)

In [9]:
from sklearn.model_selection import RandomizedSearchCV, KFold

grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid, n_iter=3, n_jobs=1)

In [11]:
grid_result = grid.fit(Images, Labels)

Epoch 1/10
1404/1404 [==============================] - 10s 7ms/step - loss: 50.3023 - accuracy: 0.4193 - val_loss: 1.9947 - val_accuracy: 0.4729
Epoch 2/10
1404/1404 [==============================] - 10s 7ms/step - loss: 0.6590 - accuracy: 0.8129 - val_loss: 2.3656 - val_accuracy: 0.4938
Epoch 3/10
1404/1404 [==============================] - 10s 7ms/step - loss: 0.3085 - accuracy: 0.9206 - val_loss: 3.0699 - val_accuracy: 0.4643
Epoch 4/10
1404/1404 [==============================] - 10s 7ms/step - loss: 0.4371 - accuracy: 0.8992 - val_loss: 3.6869 - val_accuracy: 0.4558
Epoch 5/10
1404/1404 [==============================] - 10s 7ms/step - loss: 0.6087 - accuracy: 0.8630 - val_loss: 3.0759 - val_accuracy: 0.4267
Epoch 6/10
1404/1404 [==============================] - 10s 7ms/step - loss: 0.4994 - accuracy: 0.8887 - val_loss: 2.9821 - val_accuracy: 0.4562
Epoch 7/10
1404/1404 [==============================] - 10s 7ms/step - loss: 0.3649 - accuracy: 0.9152 - val_loss: 3.8261 - val_a

In [12]:
print(grid_result.best_params_)

{'opt': 'adam', 'nn2': 200, 'nn1': 32, 'lr': 0.01}


In [13]:
best_model = grid_result.best_estimator_.model

In [14]:
test, test_labels = get_images(testDIR)
test = np.array(test)
test_labels = np.array(test_labels)

In [15]:
best_model.evaluate(test,test_labels)

94/94 [==============================] - 1s 6ms/step - loss: 9.3592 - accuracy: 0.3927


[9.359158515930176, 0.39266666769981384]